### Parse NSF Award Amount per Year

Use StartDate and EndDate to calculate the duration of the grant and AwardedAmountToDate to calculate the award amount
 - create a dayly time series
 - distribute the total amount of award across business days
 - aggregate daily amounts by year
 
 Total: 1,853,721,544

In [3]:
import numpy as np
import pandas as pd
import itertools

In [4]:
data = pd.read_csv("Data/nsf-1998-2017-all.csv", encoding = "latin1", sep=',')

In [5]:
data.head()

,AwardNumber,Title,NSFOrganization,Program(s),StartDate,LastAmendmentDate,PrincipalInvestigator,PI,AllPI,State,...,OrganizationStreet,OrganizationCity,OrganizationState,OrganizationZip,OrganizationPhone,NSFDirectorate,ProgramElementCode(s),ProgramReferenceCode(s),ARRAAmount,Abstract
0,1652052,CAREER: Active and Action-Centric Visual Under...,IIS,ROBUST INTELLIGENCE,7/1/17,8/22/18,Ali Farhadi,"Farhadi, A","Farhadi, A",WA,...,4333 Brooklyn Ave NE,Seattle,WA,981950001,2.065434e+09,CSE,7495,"1045, 7495",$0.00,This project develops technologies for visual ...
1,1741960,Spokane Falls Community College Engineering Sc...,DUE,S-STEM:SCHLR SCI TECH ENG&MATH,11/15/17,10/30/17,Mark Gorski,"Gorski, M","Gorski, M|Cripe, G|Robinson, S",WA,...,"501 N. Riverpoint Blvd., Suite 1",Spokane,WA,992176000,5.094345e+09,EHR,1536,"9178, SMET",$0.00,Spokane Falls Community College (SFCC) will pu...
2,1743772,PIRE: Science of Design for Societal-Scale Cyb...,OISE,"S&CC: Smart & Connected Commun, IIS SPECIAL PR...",10/1/17,8/30/17,Janos Sztipanovits,"Sztipanovits, J","Sztipanovits, J|Sastry, SS|Tomlin, C|Hess, D|K...",TN,...,Sponsored Programs Administratio,Nashville,TN,372350002,6.153223e+09,O/D,"033Y, 7484, 7742","042Z, 5936",$0.00,"PI: Janos Sztipanovits, Vanderbilt University<..."
3,1652588,CAREER: Safe and Transformative Robotic Interv...,IIS,ROBUST INTELLIGENCE,9/1/17,6/19/18,Daniel Rucker,"Rucker, D","Rucker, D",TN,...,1 CIRCLE PARK,KNOXVILLE,TN,379960003,8.659743e+09,CSE,7495,"1045, 7495",$0.00,Robotic devices are currently being used in me...
4,1653322,SCH: CAREER: Co-Robotic Ultrasound Sensing in...,IIS,"National Robotics Initiative, Smart and Connec...",9/1/17,7/22/17,Emad Boctor Mikhail,"Mikhail, EB","Mikhail, EB",MD,...,1101 E 33rd St,Baltimore,MD,212182686,4.439972e+09,CSE,"8013, 8018","1045, 8018, 8086",$0.00,"Ultrasound imaging, while frequently used in h..."


In [8]:
#list(data.columns)

### Daily Aggregation

In [70]:
#year = np.empty()
days = []
money = []

for index, row in data.iterrows():
    amount = row['AwardedAmountToDate'].replace('$','').replace(",",'').rstrip() 
    #print(row['AwardedAmountToDate'])
    start = pd.to_datetime(row['StartDate'])
    end = pd.to_datetime(row['EndDate'])
    
    #print(start)
    #print(end)
   
   # print(amount)
    d_range = np.array(pd.date_range(start, end)) # for business days pd.bdate_range(start, end)
   # print(len(d_range)) # days
    #print(float(amount)/len(d_range))
    row['days'] = int(float(amount)/len(d_range))
   # amount_daily = np.repeat(int(float(amount)/len(d_range)), len(d_range))  # loosing 100 dollars on decimals
    amount_daily = np.repeat(float(amount)/len(d_range), len(d_range))
    #print(amount_daily)
    #amount_monthly = np.repeat(int(float(amount)/len(d_range)), len(d_range))
    days.append(d_range)
    money.append(amount_daily)

merged_days = list(itertools.chain.from_iterable(days))
merged_money = list(itertools.chain.from_iterable(money))
y = np.array(merged_days)
m = np.array(merged_money)
df = pd.DataFrame({'year': y, 'amount' : m})

In [47]:
#df.to_csv('daily_nsf_amount.csv', index = False)

In [38]:
sum(df.amount) # total  1853721543.994824

1853721544.0088081

In [71]:
aggregate = df.groupby('year',as_index=False).agg(
    {
           'amount': 'sum' # count

                       }
)

In [72]:
aggregate['y'] = pd.DatetimeIndex(aggregate.iloc[:,0]).year

In [54]:
#aggregate['m'] = pd.DatetimeIndex(aggregate.iloc[:,0]).month  for testing

In [73]:
sum(aggregate.amount) #1853721544.000005

1853721543.9999852

In [74]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [76]:
aggregate_year = aggregate.groupby(['y'],as_index=False).agg({
    'amount':'sum'
})

In [79]:
aggregate_year.to_csv('Data/amount-daily-parse.csv', index=False)

In [77]:
sum(aggregate_year.amount)

1853721544.000001

In [58]:
#year = np.empty()
year = []
money = []

for index, row in data.iterrows():
    amount = row['AwardedAmountToDate'].replace('$','').replace(",",'').rstrip() 

    start = pd.to_datetime(row['StartDate'])
    end = pd.to_datetime(row['EndDate'])
    d_range = np.array(pd.date_range(start, end,freq='M'))
    amount_monthly = np.repeat(int(float(amount)/len(d_range)), len(d_range))
    year.append(d_range)
    money.append(amount_monthly)

merged_year = list(itertools.chain.from_iterable(year))
merged_money = list(itertools.chain.from_iterable(money))
y = np.array(merged_year)
m = np.array(merged_money)
df = pd.DataFrame({'year': y, 'amount' : m})


In [98]:
aggregate = df.groupby('year',as_index=False).agg(
    {
           'amount': 'sum' # count

                       }
)

In [101]:
#aggregate.iloc[:,0]

In [102]:
aggregate['y'] = pd.DatetimeIndex(aggregate.iloc[:,0]).year

In [104]:
aggregate_year = aggregate.groupby('y',as_index=False).agg({
    'amount':'sum'
})

In [68]:
#aggregate_year

In [45]:
aggregate_year.to_csv('Data/amount-daily-parse.csv', index=False)